In [ ]:
%%time
##在baseline_1基础上进行全局转化率平滑系数计算##
##在baseline_2基础上添加nunique特征
#在baseline_4基础上加feature_2和feature_3
#在baseline_5基础上加feature的四则运算
#!mkdir /home/admin/jupyter/env
#!pip install --prefix=/home/admin/jupyter/env lightgbm
import sys
sys.path.append('env/lib/python3.6/site-packages/')
import os
import json
import zipfile
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import time
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
start = time.time()
import jieba

##读取数据

train = pd.read_table('Demo/DataSets/oppo_data_ronud2_20181107/data_train.txt', 
        names= ['prefix','query_prediction','title','tag','label'],quoting=3,
                    header= None, encoding='utf-8').astype(str)
test = pd.read_table('Demo/DataSets/oppo_round2_test_B/oppo_round2_test_B.txt', 
        names= ['prefix','query_prediction','title','tag','label'], quoting=3,header= None, encoding='utf-8').astype(str)
valid = pd.read_table('Demo/DataSets/oppo_data_ronud2_20181107/data_vali.txt', 
        names= ['prefix','query_prediction','title','tag','label'],quoting=3, header= None, encoding='utf-8').astype(str)


##去除异常值
train = train[(train['label']=='0')|(train['label']=='1')].reset_index()
del train['index']

##打印数据形状、缺失情况、清理异常值
print(train.shape)
print(valid.shape)
print(test.shape)
test['label'] = '-1'
train.replace('nan',np.nan,inplace=True)
valid.replace('nan',np.nan,inplace=True)
test.replace('nan',np.nan,inplace=True)
print(train.isnull().sum())
print(valid.isnull().sum())
print(test.isnull().sum())
train['query_prediction'].fillna('{}',inplace=True)
test['query_prediction'].fillna('{}',inplace=True)
valid['query_prediction'].fillna('{}',inplace=True)
train['title'].fillna('-1',inplace=True)
def getSimilar(st1,st2):
    num = 0
    for s in st2:
        if s in st1:
            num+=1
    return num/len(st1)
# query 描述
def getStatList2(s):
    dct = json.loads(s)
    se = pd.Series(dct).astype(float)
    lst = se.describe().values.tolist()
    lst.append(sum(se))
    return lst
def getFeature2(data): 
    df = pd.DataFrame()
    df['query_prediction'] = data['query_prediction'].drop_duplicates().values
    arr = df['query_prediction'].apply(getStatList2)
    cols = []
    for co in pd.Series(range(100)).describe().index.tolist():
        cols.append('query_describe_'+co)
    cols.append('query_sum')
    df = pd.concat([df,pd.DataFrame(arr.values.tolist(),columns=cols)],axis=1)
    return pd.merge(data,df,how='left',on='query_prediction')[cols]
## prefix、title与 query交互
def getStatList3(s,st):
    dct = json.loads(s)
    l = []
    for k in dct.keys():
        l.append(float(dct[k])*getSimilar(k,st))
    lst = pd.Series(l).describe().values.tolist()
    lst.append(sum(l))
    if st in dct:
        lst.append(float(dct[st]))
    else:
        lst.append(np.nan)
    return lst
def getFeature3(data): 
    df = data[['query_prediction','title']].drop_duplicates().reset_index()
    del df['index']
    arr1 = df.apply(lambda row:getStatList3(row['query_prediction'], row['title']),axis = 1)
    cols1 = []
    for co in pd.Series(range(100)).describe().index.tolist():
        cols1.append('title_query_similar_'+co)
    cols1.append('title_query_similar_sum')
    cols1.append('title_query_prob')
    df1 = pd.concat([df,pd.DataFrame(arr1.values.tolist(),columns=cols1)],axis=1)
    
    
    df = data[['query_prediction','prefix']].drop_duplicates().reset_index()
    del df['index']
    arr2 = df.apply(lambda row:getStatList3(row['query_prediction'], row['prefix']),axis = 1)
    cols2 = []
    for co in pd.Series(range(100)).describe().index.tolist():
        cols2.append('prefix_query_similar_'+co)
    cols2.append('prefix_query_similar_sum')
    cols2.append('prefix_query_prob')
    df2 = pd.concat([df,pd.DataFrame(arr2.values.tolist(),columns=cols2)],axis=1)
    
    data = pd.merge(data,df1,how='left',on=['query_prediction','title'])
    data = pd.merge(data,df2,how='left',on=['query_prediction','prefix'])
    return data[cols1+cols2]
try:
    valid_feature = pd.read_csv('valid_query_describe.csv')
except:
    valid_feature = getFeature2(valid)
    valid_feature.to_csv('valid_query_describe.csv',index=False)
try:
    test_feature = pd.read_csv('test_query_describe.csv')
except:
    test_feature = getFeature2(test)
    test_feature.to_csv('test_query_describe.csv',index=False)

try:
    train_feature = pd.read_csv('train_query_describe.csv')
except:
    train_feature = getFeature2(train)
    train_feature.to_csv('train_query_describe.csv',index=False)
##############

try:
    valid_feature = pd.read_csv('valid_query_cross.csv')
except:
    valid_feature = getFeature3(valid)
    valid_feature.to_csv('valid_query_cross.csv',index=False)
try:
    test_feature = pd.read_csv('test_query_cross.csv')
except:
    test_feature = getFeature3(test)
    test_feature.to_csv('test_query_cross.csv',index=False)

try:
    train_feature = pd.read_csv('train_query_cross.csv')
except:
    train_feature = getFeature3(train)
    train_feature.to_csv('train_query_cross.csv',index=False)
###############################################################################################################################
##合并数据与相似度描述特征
# 合并到总的数据集
train['part'] = 1
valid['part'] = 2
test['part'] = 3

data = pd.concat([train,valid,test],axis=0,ignore_index=True)
data['label'] = data['label'].apply(lambda x: int(x))
train_index = data[data['label']>=0].index.tolist()
test_index =  data[data['label']==-1].index.tolist()
train_y = data.loc[train_index]['label']

def get_cut_feature(dt):
    df = pd.DataFrame()
    for item in ['prefix', 'title']:
        print(item)
        stat = pd.DataFrame()
        stat[item] = dt[item].drop_duplicates().values
        stat[item+'_cut'] = stat[item].apply(lambda x:' '.join(jieba.cut(x, cut_all=False)))
        df[item+'_cut'] = pd.merge(dt,stat,how='left',on=item)[item+'_cut']
    stat = pd.DataFrame()
    item = 'query_prediction'
    print(item)
    stat[item] = dt[item].drop_duplicates().values
    def getList(x):
        dct = json.loads(x)
        lst = []
        for k in dct.keys():
            lst.extend(jieba.cut(k,cut_all=False))
        return ' '.join(lst)
    stat['query_prediction_cut'] = stat['query_prediction'].apply(getList)
    df[item+'_cut'] = pd.merge(dt,stat,how='left',on=item)[item+'_cut']
    return df
df = get_cut_feature(data)
cntv=CountVectorizer()
vector_feature = ['prefix_cut','query_prediction_cut','title_cut']


train_sp = pd.DataFrame()
test_sp = pd.DataFrame()
for feature in vector_feature:
    print(feature)
    cntv.fit(df[feature])
    train_sp = sparse.hstack((train_sp,cntv.transform(df.loc[train_index][feature]))).tocsr()
    test_sp = sparse.hstack((test_sp,cntv.transform(df.loc[test_index][feature]))).tocsr()
del df
#####
def do_len(x):
    try:
        return len(x)
    except:
        return len(str(x))
data['prefix_len'] = data.prefix.apply(do_len)
data['title_len'] = data.title.apply(do_len)




valid_feature = pd.read_csv('valid_query_describe.csv')
test_feature = pd.read_csv('test_query_describe.csv')
train_feature = pd.read_csv('train_query_describe.csv')
data_feature = pd.concat([train_feature,valid_feature,test_feature],axis=0,ignore_index=True)
data = pd.concat([data,data_feature],axis=1)  

valid_feature = pd.read_csv('valid_query_cross.csv')
test_feature = pd.read_csv('test_query_cross.csv')
train_feature = pd.read_csv('train_query_cross.csv')
data_feature = pd.concat([train_feature,valid_feature,test_feature],axis=0,ignore_index=True)
data = pd.concat([data,data_feature],axis=1)  

del train
del test
del valid
del data_feature
del train_feature
del valid_feature
del test_feature


data['prefix_title_similar'] = data.apply(lambda row:getSimilar(row['prefix'], row['title']),axis = 1)
data['title_prefix_similar'] = data.apply(lambda row:getSimilar(row['title'], row['prefix']),axis = 1)


data['prefix_length'] = data['prefix'].apply(lambda x:len(x))
data['title_length'] = data['title'].apply(lambda x:len(x))
data['ratio_length'] = data['prefix_length']/data['title_length']



type_feature = ['prefix','query_prediction', 'title', 'tag']

### labelencoder
print('labelencodering...')
for feature in type_feature:
    se = pd.Series(data[feature].value_counts().index)
    se = pd.Series(se.index,index=se.values)
    data[feature] = data[feature].map(se)

### click
print('click...')
n = len(type_feature)
data['cnt']=1
for i in range(n):
    col_name = "cnt_click_of_"+type_feature[i]
    se = data[type_feature[i]].map(data[type_feature[i]].value_counts())
    data[col_name] =se.values

for i in range(n):
    for j in range(n-i-1):
        col_name = "cnt_click_of_"+type_feature[i+j+1]+"_and_"+type_feature[i]
        s = time.time()
        se = data.groupby([type_feature[i],type_feature[i+j+1]])['cnt'].sum()
        dt = data[[type_feature[i],type_feature[i+j+1]]]
        se = (pd.merge(dt,se.reset_index(),how='left',
                        on=[type_feature[i],type_feature[j+i+1]]).sort_index()['cnt'].fillna(value=0)).astype(int)
        data[col_name] = se.values
        data[col_name+'_info'] = data["cnt_click_of_"+type_feature[i]]*data["cnt_click_of_"+type_feature[i+j+1]]/data[col_name]
### ratio
print('ratio...')
for i in range(n):
    for j in range(n):
        if i!=j:
            col_name = "ratio_click_of_"+type_feature[j]+"_in_"+type_feature[i]
            se = data.groupby([type_feature[i],type_feature[j]])['cnt'].sum()
            dt = data[[type_feature[i],type_feature[j]]]
            cnt = data[type_feature[i]].map(data[type_feature[i]].value_counts())
            try:
                data[col_name] = ((pd.merge(dt,se.reset_index(),how='left',on=[type_feature[i],type_feature[j]]).sort_index()['cnt'].fillna(value=0)/cnt)*100).astype(int).values
            except:
                pass            
del data['cnt']

print('nunique...')
def get_nunique_feature(type_feature):
    df = pd.DataFrame()
    n = len(type_feature)
    for i in range(n):
        for j in range(n):
            if i!=j:
                try:
                    col_name = "nunique_"+type_feature[j]+"_in_"+type_feature[i]
                    se = data.groupby([type_feature[i]])[type_feature[j]].value_counts()
                    se = pd.Series(1,index=se.index).sum(level=type_feature[i])
                    df[col_name] = (data[type_feature[i]].map(se)).values
                except:
                    pass
    return df
df = get_nunique_feature(type_feature)
data = pd.concat([data,df],axis=1)

### cvr
print('cvr...')
def get_cvr_smooth_feature_total(type_feature):
    from smooth import HyperParam
    from sklearn.model_selection import StratifiedKFold
    train = data.loc[train_index]
    train_y = train['label']
    n = len(type_feature)
    df_ = pd.DataFrame()
    #######################
    n_parts = 6
    skf = StratifiedKFold(n_splits=n_parts-1,random_state=20,shuffle=True)
    data['n_parts'] = -1
    num = 1
    for trainIndex,testIndex in skf.split(train,train_y):
        data.loc[testIndex,'n_parts'] = num
        num+=1
    data.loc[data['label']==-1,'n_parts']=6
    #####################
    rate = [1,1,1,1,1,0.8]
    ###################
    for co in type_feature:
        col_name = 'cvr_of_'+co
        se =pd.Series()
        hp1 = []
        hp2 = []
        for k in range(1,n_parts+1):
            df = data[data['n_parts']==k][[co]]
            temp = data[(data['n_parts']!=k)&(data['label']>=0)&(data['part']==1)][[co,'label']].groupby(co)['label'].agg({co + '_click': 'sum', co + '_count': 'count'})
            # smooth
            temp[co + '_count'] = temp[co + '_count']*rate[k-1]
            temp[co + '_click'] = temp[co + '_click']*rate[k-1]
            hp1.extend(temp[co + '_count'].values.tolist())
            hp2.extend(temp[co + '_click'].values.tolist())
        HP = HyperParam(1, 1)
        HP.update_from_data_by_moment(np.array(hp1), np.array(hp2))
        a = HP.alpha
        b = HP.beta
        
        for k in range(1,n_parts+1):
            df = data[data['n_parts']==k][[co]]
            temp = data[(data['n_parts']!=k)&(data['label']>=0)&(data['part']==1)][[co,'label']].groupby(co)['label'].agg({co + '_click': 'sum', co + '_count': 'count'})
            # smooth
            temp[co + '_count'] = temp[co + '_count']*rate[k-1]
            temp[co + '_click'] = temp[co + '_click']*rate[k-1]
            temp[co + '_ctr_smooth'] = (temp[co + '_click'] + a) / (
                    temp[co + '_count'] + a+b)           
            se = se.append(pd.Series(df[co].map(temp[co + '_ctr_smooth']).values, index = df.index))
        df_[col_name+'_smooth'] = pd.Series(data.index).map(se)
        
        
    for i in range(n):
        for j in range(n-i-1):
            col_name = 'cvr2_of_'+type_feature[i]+"_and_"+type_feature[i+j+1]
            se = pd.Series()
            hp1 = []
            hp2 = []
            for k in range(1,n_parts+1):               
                temp = data[(data['n_parts']!=k)&(data['label']>=0)&(data['part']==1)].groupby([type_feature[i],type_feature[i+j+1]])['label'].agg({col_name + '_click': 'sum', col_name + '_count': 'count'})
                temp[col_name + '_count'] = temp[col_name + '_count']*rate[k-1]
                temp[col_name + '_click'] = temp[col_name + '_click']*rate[k-1]
                hp1.extend(temp[col_name + '_count'].values.tolist())
                hp2.extend(temp[col_name + '_click'].values.tolist())
            # smooth
            HP = HyperParam(1, 1)
            HP.update_from_data_by_moment(np.array(hp1), np.array(hp2))
            a = HP.alpha
            b = HP.beta
            for k in range(1,n_parts+1):               
                temp = data[(data['n_parts']!=k)&(data['label']>=0)&(data['part']==1)].groupby([type_feature[i],type_feature[i+j+1]])['label'].agg({col_name + '_click': 'sum', col_name + '_count': 'count'})

                temp[col_name + '_count'] = temp[col_name + '_count']*rate[k-1]
                temp[col_name + '_click'] = temp[col_name + '_click']*rate[k-1]
                temp[col_name + '_ctr_smooth'] = (temp[col_name + '_click'] + a) / (
                        temp[col_name + '_count'] + a+b)
                dt = data[data['n_parts']==k][[type_feature[i],type_feature[i+j+1]]]
                dt.insert(0,'index',list(dt.index))
                dt = pd.merge(dt,temp[col_name + '_ctr_smooth'].reset_index(),how='left',on=[type_feature[i],type_feature[i+j+1]])
                se = se.append(pd.Series(dt[col_name + '_ctr_smooth'].values,index=list(dt['index'].values)))
            df_[col_name+'_smooth'] = pd.Series(data.index).map(se).values
    del data['n_parts']
    return df_
df = get_cvr_smooth_feature_total(type_feature)
data = pd.concat([data,df],axis=1)
del data['part']
################




train_x = data.loc[train_index]

test_x = data.loc[test_index].reset_index()
del test_x['index']


train_y = train_x.pop('label')
del test_x['label']


train_x = sparse.hstack((train_x,train_sp)).tocsr()

test_x = sparse.hstack((test_x,test_sp)).tocsr()

def searchBestCut(ytrue,ypre):
    lst = []
    for i in range(10,81,1):
        lst.append(f1_score(ytrue,(pd.Series(ypre)>=i*0.01).astype(int).values))
    se = pd.Series(lst,index=range(10,81,1)).sort_values()
    return se


params_default_lgb = {
        'num_leaves':80, 
        'learning_rate':0.1, 
    'boosting':'gbdt',
    'min_child_samples':20,
    
    'bagging_fraction':1, 
    'bagging_freq':1,
    'feature_fraction':1, 
     'max_depth':-1,
    'reg_alpha':2,
    'reg_lambda':5, 
    'metric':'binary_logloss',
    'objective':'binary'
}
print(train_x.shape)
print(test_x.shape)
NBR = 20000
VBE = 100
ESR = 200

EVAL_RESULT = {}
train_part = lgb.Dataset(train_x[:-50000],label=train_y[:-50000])
evals = lgb.Dataset(train_x[-50000:],label=train_y[-50000:])
bst = lgb.train(params_default_lgb,train_part, 
      num_boost_round=NBR, valid_sets=[train_part,evals], 
      valid_names=['train','evals'],  early_stopping_rounds=ESR,
      evals_result=EVAL_RESULT, verbose_eval=VBE)
lst = EVAL_RESULT['evals']['binary_logloss']
best_iter = lst.index(min(lst))+1

evals_ypre =  bst.predict(train_x[-50000:],num_iteration = best_iter)
se = searchBestCut(train_y[-50000:],evals_ypre)       
cut,best_score = (se.index[-1])*0.01,se.values[-1]

train_part = lgb.Dataset(train_x,label=train_y)
bst = lgb.train(params_default_lgb,train_part, 
      num_boost_round=best_iter)
test_ypre =  bst.predict(test_x,num_iteration = best_iter)
res = pd.Series(test_ypre)

filename = 'result/1202.csv'
(res>=cut).astype(int).to_csv(filename,index=False)

print((res>=cut).astype(int).value_counts())
print(filename)
newZip = zipfile.ZipFile('submission/1202.zip', 'w')
newZip.write(filename, compress_type=zipfile.ZIP_DEFLATED)
newZip.close()